In [1]:
import os

files = os.listdir('./DataSet/')

In [5]:
import json
import numpy as np

t_stamps, tot_posts = {}, 0

for year in range(2018, 2024):
    y_files = [f for f in files if str(year) in f]
    # print(year, len(y_files))
    np_list, tok_list, stp_list = [], [], []
    for yf in y_files:
        with open(yf, 'r') as f:
            data = json.load(f)
            vals, stamps = list(data.values()), list(data.keys())
            nr_toks = [len(v.split()) for v in vals]
            nr_posts = len(vals)
            tot_posts += nr_posts
            np_list.append(nr_posts)
            tok_list.append(np.mean(nr_toks))
            stp_list.append(stamps)
    avg_np = np.mean(np_list)
    avg_tok = np.mean(tok_list)
    t_stamps[year] = stp_list
    print(year, avg_np, avg_tok)

2018 231.85 16.23036360486034
2019 166.15 19.02990359980832
2020 123.05882352941177 21.677227784549324
2021 108.38095238095238 22.37070651930522
2022 119.77272727272727 24.420860912482024
2023 140.95238095238096 25.77170418981877


In [6]:
tot_posts

17923

In [20]:
from datetime import datetime


def to_dt(stamp):
    return datetime.strptime(stamp, '%Y-%m-%dT%H:%M:%SZ')

In [37]:
from scipy import stats

for year in t_stamps:
    stamp_lists = t_stamps[year]
    diffs = []
    for sl in stamp_lists:
        sl_diffs = []
        for i in range(len(sl)):
            if i == 0: continue
            prev, cur = to_dt(sl[i-1]), to_dt(sl[i])
            diff = prev - cur
            sl_diffs.append(diff.seconds)
        sl_diffs = np.array(sl_diffs)
        z = np.abs(stats.zscore(sl_diffs))
        outliers = list(np.where(z > 2))
        for o in sorted(outliers, reverse=True):
            sl_diffs = np.delete(sl_diffs, o)
        diffs += [sld for sld in sl_diffs]
    print(year, np.mean(diffs))

2018 30.6213936972925
2019 39.22639149468418
2020 64.45417925478348
2021 83.17687701520038
2022 84.66136724960255
2023 72.06942496493689


In [2]:
f2018 = [f for f in files if '2018' in f]
len(f2018)

20